In [20]:
from typing import TypedDict, Annotated
from langgraph.graph import add_messages, StateGraph, END
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage
from dotenv import load_dotenv
# In-memory Checkpointer
from langgraph.checkpoint.memory import MemorySaver

In [21]:
load_dotenv()

True

In [22]:
memory=MemorySaver()

In [23]:
llmLlama=ChatGroq(model="llama-3.1-8b-instant")

In [24]:
class BasicChatState(TypedDict):
    messages: Annotated[list, add_messages]

In [25]:
def chatbot(state: BasicChatState):
    return {
        "messages":[llmLlama.invoke(input=state["messages"])]
    }

In [26]:
graph=StateGraph(state_schema=BasicChatState)

In [27]:
graph.add_node(node="chatbot",action=chatbot)

In [28]:
graph.add_edge(start_key="chatbot",end_key=END)

In [29]:
graph.set_entry_point(key="chatbot")

In [30]:
app=graph.compile(checkpointer=memory)

In [34]:
config={"configurable":{"thread_id":1}}

In [35]:
response1=app.invoke(input={"messages":HumanMessage(content="Hi I am Harish")},config=config)

In [36]:
response1

{'messages': [HumanMessage(content='Hi I am Harish', additional_kwargs={}, response_metadata={}, id='992d4954-5a0d-4829-b17c-cfadca5411dd'),
  AIMessage(content="Hello Harish, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 40, 'total_tokens': 67, 'completion_time': 0.036, 'prompt_time': 0.007631657, 'queue_time': 0.05180164, 'total_time': 0.043631657}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_8d1c73bd4c', 'finish_reason': 'stop', 'logprobs': None}, id='run-ba6ce960-6735-4b84-8102-ba09e4003cdc-0', usage_metadata={'input_tokens': 40, 'output_tokens': 27, 'total_tokens': 67})]}

In [37]:
response2=app.invoke(input={"messages":HumanMessage(content="What's my name?")},config=config)

In [38]:
response2

{'messages': [HumanMessage(content='Hi I am Harish', additional_kwargs={}, response_metadata={}, id='992d4954-5a0d-4829-b17c-cfadca5411dd'),
  AIMessage(content="Hello Harish, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 40, 'total_tokens': 67, 'completion_time': 0.036, 'prompt_time': 0.007631657, 'queue_time': 0.05180164, 'total_time': 0.043631657}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_8d1c73bd4c', 'finish_reason': 'stop', 'logprobs': None}, id='run-ba6ce960-6735-4b84-8102-ba09e4003cdc-0', usage_metadata={'input_tokens': 40, 'output_tokens': 27, 'total_tokens': 67}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='1e5238e1-41c4-4e00-8aa6-25213732b238'),
  AIMessage(content='Your name is Harish.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7

In [39]:
# get snapshot of the latest checkpoint
snapshot=app.get_state(config=config)

In [40]:
snapshot

StateSnapshot(values={'messages': [HumanMessage(content='Hi I am Harish', additional_kwargs={}, response_metadata={}, id='992d4954-5a0d-4829-b17c-cfadca5411dd'), AIMessage(content="Hello Harish, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 40, 'total_tokens': 67, 'completion_time': 0.036, 'prompt_time': 0.007631657, 'queue_time': 0.05180164, 'total_time': 0.043631657}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_8d1c73bd4c', 'finish_reason': 'stop', 'logprobs': None}, id='run-ba6ce960-6735-4b84-8102-ba09e4003cdc-0', usage_metadata={'input_tokens': 40, 'output_tokens': 27, 'total_tokens': 67}), HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='1e5238e1-41c4-4e00-8aa6-25213732b238'), AIMessage(content='Your name is Harish.', additional_kwargs={}, response_metadata={'token_usage': {'comple

In [41]:
while True:
    user_input=input("User: ")
    if (user_input in ["exit","end"]):
        break
    else:
        result=app.invoke(input={
            "messages":HumanMessage(content=user_input)
        },config=config)
        print(result['messages'][-1].content)

User:  What's my name?


Your name is Harish.


User:  I am from Noida


Noida is a city in the National Capital Region (NCR) of India, near Delhi. It's a popular area for IT and business hubs. What brings you to this chat, Harish?


User:  I want to know about IPL


The Indian Premier League (IPL) is a popular professional Twenty20 cricket league in India. It was founded in 2008 and has since become one of the most-watched and followed sports leagues in the world.

Here's a brief overview:

- The IPL features 10 teams, each representing a different city in India.
- The league is played over several weeks in March and April, with each team playing the others in a round-robin format.
- The top four teams then qualify for the playoffs, which include the Qualifier 1 and 2, and the Eliminator.
- The final match is played between the winners of Qualifier 1 and 2.

Some of the most successful teams in the IPL include:

- Mumbai Indians (5 titles)
- Chennai Super Kings (4 titles)
- Kolkata Knight Riders (2 titles)
- Rajasthan Royals (1 title)
- Sunrisers Hyderabad (1 title)

Some of the top players in the IPL include:

- Virat Kohli (Royal Challengers Bangalore)
- Rohit Sharma (Mumbai Indians)
- MS Dhoni (Chennai Super Kings)
- Ravindra Jadeja (Chennai Su

User:  What is it's full form?


The full form of IPL is Indian Premier League.


User:  exit


In [ ]:
while True:
    user_input=input("User: ")
    if (user_input in ["exit","end"]):
        break
    else:
        result=app.invoke(input={
            "messages":HumanMessage(content=user_input)
        },config=config)
        print(result['messages'][-1].content)

User:  What's my name?


Your name is Harish.
